<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/4_pose_estimation/GC4_0_make_folders_and_data_downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  第4章 姿勢推定の準備ファイル

- 本ファイルでは、第4章で使用するフォルダの作成とファイルのダウンロードを行います。

---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

Change to the JST notation.


In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

Start mounting your Google Drive.
Mounted at /content/drive
/content/drive/My Drive
Move to the working directory.
/content/drive/My Drive/202107_Tool-A/Work600
total 4
drwx------ 2 root root 4096 Jul 31 09:41 pytorch_advanced


---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

OK. Alreadly git cloned. You can go.


In [ ]:
!ls

pytorch_advanced


In [ ]:
%cd "pytorch_advanced"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced


In [ ]:
!ls

1_image_classification	 7_nlp_sentiment_transformer
2_objectdetection	 8_nlp_sentiment_bert
3_semantic_segmentation  9_video_classification_eco
4_pose_estimation	 etc
5_gan_generation	 LICENSE
6_gan_anomaly_detection  README.md


In [ ]:
%cd "4_pose_estimation"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation


In [ ]:
!ls

4-2_DataLoader.ipynb	     4-7_OpenPose_inference.ipynb	    utils
4-3-4_NetworkModel.ipynb     data				    weights
4-5_TensorBoardX.ipynb	     make_folders_and_data_downloads.ipynb
4-6_OpenPose_training.ipynb  tbX


---
# Utils, Data, and Weights

[U-Tsukuba] Extra remarks:

Working directories on the run are:

* /root/utils (*.py) 
* /root/data (val2014.zip, COCO.json, mask.tar.gz, hit-1407826_640.jpg) 
* /root/weights  (pose_model_scratch.pth)

val2014.zip and mask.tar.gz will be expanded there when needed.

By then, files are stored at /content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation.

---
# Directories

At /content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation :

In [ ]:
%cd '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation'

In [ ]:
import os
import urllib.request
import zipfile
import tarfile

In [ ]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
# フォルダ「weights」が存在しない場合は作成する
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [ ]:
!ls -l data weights

data:
total 6936527
-rw------- 1 root root  427299390 Aug  1 14:23 COCO.json
-rw------- 1 root root      98237 Jul 31 09:42 hit-1407826_640.jpg
-rw------- 1 root root   30591896 Aug  1 12:52 mask.tar.gz
-rw------- 1 root root 6645013297 Aug  1 12:49 val2014.zip

weights:
total 204378
-rw------- 1 root root 209282651 Aug  1 12:52 pose_model_scratch.pth


---
# val2014.zip (MS COCO)

In [ ]:
# MSCOCOの2014 Val images [41K/6GB]をダウンロード
# 6GBのダウンロードなので時間がかかります（3分弱）
# ２回目以降は再ダウンロードしません

url =  "http://images.cocodataset.org/zips/val2014.zip"
target_path = os.path.join(data_dir, "val2014.zip") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)    


In [ ]:
!ls -l data

total 6936527
-rw------- 1 root root  427299390 Aug  1 14:23 COCO.json
-rw------- 1 root root      98237 Jul 31 09:42 hit-1407826_640.jpg
-rw------- 1 root root   30591896 Aug  1 12:52 mask.tar.gz
-rw------- 1 root root 6645013297 Aug  1 12:49 val2014.zip


---
# tensorboyさんのGitHubから必要なデータを入手

## 1. COCO.json

COCODatasetから使用しやすいようにtensorboyさんが加工したデータを使用します

https://github.com/tensorboy/pytorch_Realtime_Multi-Person_Pose_Estimation

Released under the MIT license  

https://www.dropbox.com/s/0sj2q24hipiiq5t/COCO.json?dl=0  

[U-Tsukuba] I make a local copy for automated procedure.  
Local mirror of COCO.json (407.5MB)  
https://drive.google.com/file/d/1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt

In [ ]:
# google drive downloader (for a large file)
# https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
# You should change target directory and the first two lines (FILEID and OUTFILENAME) only

# moving to the target directory
%cd data

! FILEID=1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt; \
OUTFILENAME=COCO.json; \
\
URLS="https://docs.google.com/uc?export=download&id=$FILEID"; \
echo $FILEID $OUTFILENAME $URLS; \
URLC=`wget --quiet --save-cookies /root/cookies.txt --keep-session-cookies --no-check-certificate $URLS -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'`; \
echo $URLC; \
wget --load-cookies /root/cookies.txt "https://docs.google.com/uc?export=download&confirm="$URLC"&id="$FILEID -O $OUTFILENAME ; \
rm -f /root/cookies.txt 

# just check
!ls -l

# coming back to the original directory
%cd -

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/data
1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt COCO.json https://docs.google.com/uc?export=download&id=1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt
FUDB
--2021-08-01 14:22:39--  https://docs.google.com/uc?export=download&confirm=FUDB&id=1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt
Resolving docs.google.com (docs.google.com)... 108.177.97.139, 108.177.97.102, 108.177.97.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ojgkq1mngvv4s58sn775mn6u04qf6emi/9an24dfe2savthra0dnatnnaim8nfugv/1627795350000/13943300437040148116/06255471083340725048Z/1rTf1dZpee31BKmMw7DAX9UZiQOQDuCwt?e=download [following]
--2021-08-01 14:22:39--  https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ojgkq1mngvv4s58sn775mn6u04qf6emi/9an24dfe2savthra0dnatnnaim8nfugv/1627795350000/13943300437

## 2. mask.tar.tz

画像中でアノテーション（人の関節位置の情報）がない人の部分をマスクするデータを用意します。

フォルダ「data」に以下のデータ「mask.tar.gz」を配置
マスクデータ「mask.tar.gz」

https://www.dropbox.com/s/bd9ty7b4fqd5ebf/mask.tar.gz?dl=0

[U-Tsukuba] I make a local copy for automated procedure.  
Local mirror of mask.tar.gz (29.2MB)  

https://drive.google.com/file/d/1b50ikKydibLEHKKn5EHKm76y-2RC7OqL

In [ ]:
# google drive downloader (for a large file)
# https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
# You should change target directory and the first two lines (FILEID and OUTFILENAME) only

# moving to the target directory
%cd data

! FILEID=1b50ikKydibLEHKKn5EHKm76y-2RC7OqL; \
OUTFILENAME=mask.tar.gz; \
\
URLS="https://docs.google.com/uc?export=download&id=$FILEID"; \
echo $FILEID $OUTFILENAME $URLS; \
URLC=`wget --quiet --save-cookies /root/cookies.txt --keep-session-cookies --no-check-certificate $URLS -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'`; \
echo $URLC; \
wget --load-cookies /root/cookies.txt "https://docs.google.com/uc?export=download&confirm="$URLC"&id="$FILEID -O $OUTFILENAME ; \
rm -f /root/cookies.txt 

# just check
!ls -l

# coming back to the original directory
%cd -

---
# 学習済みモデルのダウンロード (weights)

tensorboyさんによる、学習済みのOpenPoseモデル「pose_model_scratch.pth」を用意します。

フォルダ「weights」に以下の「pose_model_scratch.pth」を配置
学習済みモデル「pose_model_scratch.pth」

https://www.dropbox.com/s/5v654d2u65fuvyr/pose_model_scratch.pth?dl=0

[U-Tsukuba] I make a local copy for automated procedure.  
Local mirror of pose_model_scratch.pth (199.6MB)  

https://drive.google.com/file/d/11N7ZfbsGxjC8LBWyhlFrWKdw_N1xbufp

In [ ]:
# google drive downloader (for a large file)
# https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
# You should change the first two lines (FILEID and OUTFILENAME) only

# moving to the target directory
%cd weights

! FILEID=11N7ZfbsGxjC8LBWyhlFrWKdw_N1xbufp; \
OUTFILENAME=pose_model_scratch.pth; \
\
URLS="https://docs.google.com/uc?export=download&id=$FILEID"; \
echo $FILEID $OUTFILENAME $URLS; \
URLC=`wget --quiet --save-cookies /root/cookies.txt --keep-session-cookies --no-check-certificate $URLS -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'`; \
echo $URLC; \
wget --load-cookies /root/cookies.txt "https://docs.google.com/uc?export=download&confirm="$URLC"&id="$FILEID -O $OUTFILENAME ; \
rm -f /root/cookies.txt 

# just check
!ls -l

# coming back to the original directory
%cd -

---
# 草野球の画像を手動でダウンロード（予め実施済）


https://pixabay.com/ja/photos/%E3%83%92%E3%83%83%E3%83%88-%E3%82%AD%E3%83%A3%E3%83%83%E3%83%81%E3%83%A3%E3%83%BC-%E9%87%8E%E7%90%83-1407826/

の640×426サイズの画像
（画像権利情報：商用利用無料、帰属表示は必要ありません）

は、フォルダ「data」の直下に置いてあります。



---
# 最終チェック

In [ ]:
%cd '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation'
!ls -l utils data weights

data:
total 6936527
-rw------- 1 root root  427299390 Aug  1 14:23 COCO.json
-rw------- 1 root root      98237 Jul 31 09:42 hit-1407826_640.jpg
-rw------- 1 root root   30591896 Aug  1 12:52 mask.tar.gz
-rw------- 1 root root 6645013297 Aug  1 12:49 val2014.zip

weights:
total 204378
-rw------- 1 root root 209282651 Aug  1 12:52 pose_model_scratch.pth


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/02. 